# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Ниже в проекте вы можете найти описание процесса преобработки входных данных от банка с помощью Python. Поиск ошибок и усовершенствание данных - важный этап при анализе данных. В данном проекте изучены наиболее вероятные ошибки в данных по каждому столбцу. Критичные ошибки, которые важны для изучения поставленных в проекте вопросов, были исправлены с помощью различных инструментов Python.

Краткое содержание проекта:
- Изучение общей информации о файле с данными.
- Изучение каждого столбца и поиск потенциальных ошибок.
- Работа над ошибками файла. Заполнение пропусков и исправление ошибок.
- Изменение типов данных.
- Обработка дубликатов.
- Лемматизация данных.
- Категоризация данных.
- Выводы и поиск ответов на основные вопросы. поставленных в проекте.

Автор проекта: 
- Виктория Кускова, 
- студентка Яндекс.Практикума, кагорты DS20,
- Февраль 2021.

## Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [53]:
#импорты библиотек и ключевые константы для проекта
import pandas as pd
import numpy as np
from pymystem3 import Mystem
m = Mystem()

In [3]:
#импорт файла с данными
data = pd.read_csv('/datasets/data.csv')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [7]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [8]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


При первичном осмотре таблицы с данными о платежеспособности клиентов мы поняли, то в ней содержится 21 525 строк и 12 столбцов, в которых содержатся числовые (типы float64 и int64) и текстовые значения (тип object). Размер файла с даннами - 2.0+ MB.

Также сразу заметно, что в некоторых столбцах есть пропуски: days_employed, total_income. В двух столбцах days_employed и children есть положительные и отрицательные значения, что требует дополнительной проверки.

Далее приступаем к более подробному анализу данных по отдельным столбцам.

### Столбец 'children'

In [3]:
#анализ столбца 'children'
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [4]:
data.groupby('children')['children'].count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Мы видим явную ошибку в строках со значениями -1 и 20 в столбце с количеством детей. Также можно проверить значение 5, по нему только 9 строк и можно просмотреть их глазами. А именно можно проверить возраст человека у которого 5 детей (он должен быть больше или равен примерно 30) и семейный статус (наличие брака в настоящем или прошлом). 

По строкам со значением -1 есть предположение, что это тоже самое, что и 1. Чтобы наверняка убедиться в этом можно проверить средний возраст и распределение по семейному статусу в этих строках.

По строкам со значением 20 нет однозначного предположения. Можно более подробно изучить эти строки: возможно среднее значение по возрасту будет близко к строкам с 1 или 2 детьми (тогда 0 в конце просто лишний, или возможно писали возраст ребенка), или если средний возраст будет около 20, то возможно в это поле продублировалось значение возраст клиента. Или это был год заполнения. Необходимо выбрать наиболее подходящую гипотезу и заменить значение. Или в крайнем случае удалить, так как это меньше 1% от общего числа строк (76/21525 = 0,35%).

In [17]:
data[data['children']==5]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3979,5,NaN,42,среднее,1,гражданский брак,1,M,сотрудник,0,NaN,на покупку своего автомобиля
4397,5,-3248.839837,36,среднее,1,женат / замужем,0,F,компаньон,0,168460.926969,операции с недвижимостью
7866,5,-773.124856,36,среднее,1,женат / замужем,0,F,сотрудник,0,48772.896208,операции с жильем
15822,5,-418.199982,31,среднее,1,женат / замужем,0,F,сотрудник,0,77552.432218,сделка с подержанным автомобилем
15916,5,-2286.262752,37,среднее,1,женат / замужем,0,F,сотрудник,0,256698.352846,покупка недвижимости
16211,5,-387.317579,35,среднее,1,гражданский брак,1,F,госслужащий,0,126102.147792,на проведение свадьбы
20452,5,-268.425464,38,НАЧАЛЬНОЕ,3,женат / замужем,0,F,сотрудник,0,212545.366151,заняться высшим образованием
20837,5,-2386.600221,35,среднее,1,женат / замужем,0,F,компаньон,0,204241.893533,жилье
21156,5,-1690.018117,59,среднее,1,женат / замужем,0,M,сотрудник,0,269068.348794,операции со своей недвижимостью


In [7]:
data.groupby('children')['dob_years'].mean()

children
-1     42.574468
 0     46.261432
 1     38.366127
 2     35.770316
 3     36.287879
 4     36.048780
 5     38.777778
 20    41.815789
Name: dob_years, dtype: float64

In [13]:
data.pivot_table(index=['children','family_status'], values='dob_years', aggfunc='count')

dob_years
children family_status                   
-1       Не женат / не замужем          5
         в разводе                      4
         вдовец / вдова                 4
         гражданский брак               5
         женат / замужем               29
 0       Не женат / не замужем       2265
         в разводе                    784
         вдовец / вдова               848
         гражданский брак            2752
         женат / замужем             7500
 1       Не женат / не замужем        449
         в разводе                    312
         вдовец / вдова                77
         гражданский брак             998
         женат / замужем             2982
 2       Не женат / не замужем         75
         в разводе                     81
         вдовец / вдова                20
         гражданский брак             344
         женат / замужем             1535
 3       Не женат / не замужем          8
         в разводе                     11
         вдовец / вдова                 6
         гражданский брак              56
         женат / замужем              249
 4       Не женат / не замужем          2
         в разводе                      1
         вдовец / вдова                 1
         гражданский брак               8
         женат / замужем               29
 5       гражданский брак               2
         женат / замужем                7
 20      Не женат / не замужем          9
         в разводе                      2
         вдовец / вдова                 4
         гражданский брак              12
         женат / замужем               49

Итого по столбцу 'children':
- строки со значением 5 не трогаем (у всех возраст больше 30, семейный статус);
- строки со значением -1 заменяем на значение 1 (средний возраст и распределние семейного положения положения похоже на строки с 1 ребенком);
- строки со значением 20: средний возраст ближе всего к людям с 1 ребенком, распределение семейного положения тоже. Поэтому заменим это значение на 1.

### Столбцы 'days_employed'  и 'total_income'

Эти два столбца мы анализируем вместе, так как есть предположение, что пустые значения в них происходят в одних и тех же случаях.

Проверим это:

In [22]:
#анализ столбца 'days_employed'
data[data['days_employed'].isnull()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Как мы видим, если смотреть только на те строки таблицы, где в столбце 'days_employed' нет значения - то во всех таких строках значение по столбцу 'total_income' тоже пропущено. Поэтому эти пропуски взаимозависимы. Возможно данные по работе по данным клиентам тянулись автоматически из определенной базы данных и произошел сбой. Или в месте сбора информации данные поля не были обязательны не заполнения.

- Возможно эти данные пропуски означают то, что человек еще не работал и его стаж равен 0, как и ежемесячный доход. Это можно проверить по столбцу 'income_type', посмотрев должности.
- Если не подтвердится гипотеза выше, то можно заменить пропуски средним значением по группам по возрасту и роду деятельности.
Так как это уже около 1% всех строк, то это очень важная группа для анализа кредитоспособности, поэтому эти пропуски нельзя удалять.

Проверим это, а также посмотрим на другие значения в этих столбцах.

In [23]:
data[data['days_employed'].isnull()].groupby('income_type')['dob_years'].mean()

income_type
госслужащий        41.231293
компаньон          40.086614
пенсионер          58.445521
предприниматель    58.000000
сотрудник          40.031674
Name: dob_years, dtype: float64

Как мы видим, пропуски явно не связаны с отсутствием стажа. Это исключает сами виды деятельности, указывающие, что люди либо сейчас, либо в прошлом были трудоустроены. Поэтому при обработке данных заменим пропуски средним значением по группам по возрасту.

Теперь посмотрим на сами данные в столбцах. При просмотре нескольких строк выше было видно, что значения то отрицательные, то нет. Так как отрицательного стажа быть не может, то нужно заменить отрицательные значения на положительные. Скорее всего опять к данным поднятулось ненужное тире, которое прекратилось в минус. 

In [27]:
data[data['days_employed'] < 0]['days_employed'].count()

15906

In [28]:
data[data['days_employed'] > 0]['days_employed'].count()

3445

In [29]:
data[data['days_employed'] == 0]['days_employed'].count()

0

Перепроверим себя: 15906 + 3445 + 0 + 2174 (пустые значения) = 21525.
Значит, никаких лишних значений у нас нет.

Также проверим что нет нереалистичных значений вылетов - проверим пятерку максимальных и минимальных значений.

In [33]:
data.sort_values(by='days_employed', ascending=False).head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба


In [31]:
data.sort_values(by='days_employed', ascending=True).head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,-17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,-16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,-16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,-16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости


Как мы видим - числа по модулю сильно расходятся. В положительных значениях это около 400 тыс, а отрицательных максимум 18 тыс. Если посмотреть сколько это в годах (делим на 365), то видим, что это 1100 лет и 50 лет. Очевидно, что в строках с положительными значениями ошибки.

Нужно проверить во всех ли и почему. Если поделить 400 тыс на 365 и на 24 (предполагаем, что это сстаж случайно перевели до часов, а не дней), то получаем примерно 45 лет, что уже соответсвует годам. Давайте посмотрим на среднее и на минимальные значения.

In [34]:
data[data['days_employed'] > 0]['days_employed'].mean()

365004.3099162686

In [39]:
data[data['days_employed'] > 0].sort_values(by='days_employed', ascending=True).head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20444,0,328728.720605,72,среднее,1,вдовец / вдова,2,F,пенсионер,0,96519.339647,покупка жилья для семьи
9328,2,328734.923996,41,высшее,0,женат / замужем,0,M,пенсионер,0,126997.497760,операции со своей недвижимостью
17782,0,328771.341387,56,среднее,1,женат / замужем,0,F,пенсионер,0,68648.047062,операции с коммерческой недвижимостью
14783,0,328795.726728,62,высшее,0,женат / замужем,0,F,пенсионер,0,79940.196752,на покупку своего автомобиля
7229,1,328827.345667,32,среднее,1,гражданский брак,1,F,пенсионер,0,122162.965695,сыграть свадьбу


In [41]:
data[data['days_employed'] > 0]['dob_years'].mean()

59.12481857764877

Видно что в этих данных с положительным и при этом завышенным значением стажа в днях следующие закономерности:
- средний возраст данной группы велик, 
- и в прошлых выгрузках тип занятости был только "пенсионер"
- даже минимальные значения в этой группе явно завышены и неправдоподобны (328 тыс дней).

Возможно ошибка допущена для клиентов-пенсионеров. Проверим как распределены значения по другим типам занятости:

In [42]:
data[data['days_employed'] > 0].groupby('income_type')['income_type'].count()

income_type
безработный       2
пенсионер      3443
Name: income_type, dtype: int64

In [43]:
data[data['days_employed'] < 0].groupby('income_type')['income_type'].count()

income_type
в декрете              1
госслужащий         1312
компаньон           4577
предприниматель        1
сотрудник          10014
студент                1
Name: income_type, dtype: int64

Как видим группы по типу занятости не пересекаются. Но ошибка также есть и для категории "безработный".
Для этих категорий мы сделаем замену значений, и поделим это еще на 24 часа. Скорее всего при получении данных стаж перевели не в дни, а сразу в часы по данным группам.

Теперь изучим столбец 'total_income':

In [46]:
#анализ столбца 'total_income'
data[data['total_income'] > 0]['total_income'].count()

19351

Перепроверим себя: 19351 + 2174 (пустые значения) = 21525. Значит, никаких лишних значений у нас нет, и все доходы положительные.

Проверим на вылеты.

In [50]:
data[data['total_income'] > 0].sort_values(by='total_income', ascending=True).head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14585,0,359219.059341,57,среднее,1,женат / замужем,0,F,пенсионер,1,20667.263793,недвижимость
13006,0,369708.589113,37,среднее,1,гражданский брак,1,M,пенсионер,0,21205.280566,заняться высшим образованием
16174,1,-3642.820023,52,Среднее,1,женат / замужем,0,M,сотрудник,0,21367.648356,приобретение автомобиля
1598,0,359726.104207,68,среднее,1,гражданский брак,1,M,пенсионер,0,21695.101789,на проведение свадьбы
14276,0,346602.453782,61,среднее,1,женат / замужем,0,F,пенсионер,0,21895.614355,недвижимость


In [53]:
data[data['total_income'] > 0].sort_values(by='total_income', ascending=False).head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12412,0,-1477.438114,44,высшее,0,женат / замужем,0,M,компаньон,0,2.265604e+06,ремонт жилью
19606,1,-2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2.200852e+06,строительство недвижимости
9169,1,-5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1.726276e+06,дополнительное образование
20809,0,-4719.273476,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1.715018e+06,покупка жилья для семьи
17178,0,-5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1.711309e+06,сыграть свадьбу


Максимальные значения неправдоподобно высоки. Возможно кто столько и зарабатывает, и возможно кредиты нужны на развития бизнеса. Но тогда неправдоподобно выглядяит, к примеру, причина кредита "сыграть свадьбу". Попробуем разобраться что тут такое.

In [59]:
data[data['total_income'] > 1000000]['total_income'].count()

25

In [60]:
data['total_income'].max()

2265604.028722744

In [61]:
data[data['total_income'] > 1000000].pivot_table(index=['income_type','purpose'], values='dob_years', aggfunc='count')

dob_years
income_type purpose                                         
компаньон   автомобиль                                     1
            жилье                                          1
            на покупку автомобиля                          1
            операции с жильем                              2
            операции с коммерческой недвижимостью          1
            операции с недвижимостью                       2
            покупка коммерческой недвижимости              1
            профильное образование                         1
            ремонт жилью                                   1
            свадьба                                        1
            строительство недвижимости                     2
            сыграть свадьбу                                1
сотрудник   дополнительное образование                     1
            операции с жильем                              1
            покупка жилья                                  1
            покупка жилья для семьи                        2
            покупка коммерческой недвижимости              1
            покупка недвижимости                           1
            покупка своего жилья                           1
            получение образования                          2

Видим, что совсем больших значений не так много, максимальное тоже выглядит реалистично.
Также тип занятости (Компаньон и Сотрудник) у топа по зарплатам соответствует представлению о больших заработках. И цели такие, как покупка комерческой недвижимости или строительство соответсвуют большим затратам.

По столбцу 'total_income' только заполняем пустоты на основе возрастных групп, причем необходимо использовать медиану, так как есть несколько больших значений, которые могут отрицательно повлиять на среднее.

### Столбец 'dob_years'

In [62]:
#анализ столбца 'dob_years'
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Значения по возрасту выглядят правдоподобно, кроме значения 0. Посмотрим много ли таких значений:

In [64]:
data[data['dob_years'] == 0]['dob_years'].count()

101

In [67]:
data[data['dob_years'] == 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,-934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,-5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,NaN,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,жилье
1898,0,370144.537021,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


К сожалению, определенной закономерности не заметно. Можно подумать над вариантом заменить 0 на средние значения исходя из групп по семейному статусу.
Это 101 строка, что уже почти половина процента от всех строк (101/21525 = 0,47%), поэтому выкидывать их не верно.

### Столбцы 'education' и 'education_id'

In [69]:
#анализ столбцов 'education' и 'education_id'
data.pivot_table(index=['education_id','education'], values='dob_years', aggfunc='count')

dob_years
education_id education                     
0            ВЫСШЕЕ                     274
             Высшее                     268
             высшее                    4718
1            СРЕДНЕЕ                    772
             Среднее                    711
             среднее                  13750
2            НЕОКОНЧЕННОЕ ВЫСШЕЕ         29
             Неоконченное высшее         47
             неоконченное высшее        668
3            НАЧАЛЬНОЕ                   17
             Начальное                   15
             начальное                  250
4            УЧЕНАЯ СТЕПЕНЬ               1
             Ученая степень               1
             ученая степень               4

Видим, что отличия только в регистре, поэтому необходимо будет привести столбец 'education' к единой форме.

### Столбцы 'family_status' и 'family_status_id'

In [70]:
#анализ столбцов 'family_status' и 'family_status_id'
data.pivot_table(index=['family_status_id','family_status'], values='dob_years', aggfunc='count')

,,dob_years
family_status_id,family_status,
0,женат / замужем,12380
1,гражданский брак,4177
2,вдовец / вдова,960
3,в разводе,1195
4,Не женат / не замужем,2813


Видим, что тут данные однообразны. Можно ничего не елать с данными столбцами, только в значении 4 возможно привести все к нижнему регистру.

### Столбцы 'gender', 'income_type', 'debt'

In [71]:
#анализ столбца 'gender'
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [75]:
data[data['gender'] == 'XNA']['gender'].count()

1

In [76]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Видим, что всего одна строка с неизвестным полом. Ей можно пренебречь и оставить не меняя, так как пол для данной задачи не нужен.

In [72]:
#анализ столбца 'income_type'
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [73]:
#анализ столбца 'debt'
data['debt'].unique()

array([0, 1])

По столбцам 'debt' и 'income_type' все ок, ничего с ними не делаем.

### Столбец 'purpose'

In [77]:
#анализ столбца 'purpose'
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Видим, что данный набор досточно неоднородный. Но можно выделить следующие основные направления:
- жилье/недвижимость
- образование
- атокредит
- свадьба

По ним нужно создать отдельный столбец и перегрупировать данные цели, делая из более однододными.

### Вывод

Подведем краткие итоги.

Таблица с данными о платежеспособности клиентов содержит 21525 строк и 12 столбцов. После детального анализа по столбцам необходмы следующие действия, которые будут реализованы в следующих шагах:

- 'children': значение -1 меняем на 1, строки со значением 20 также меняем на значение 1 (76 строк = 0,35%).
- 'days_employed' и 'total_income' содержат пропуски. Пропуски мы заменим на среднее и медиану значений внутри групп по возрасту. Также положительные значения 'days_employed' необходимо поделить на 24, а отрицательные перевести в положительные по модулю.
- 'dob_years': строки со значением 0 (101 строка = 0,47%) заменим на среднее значение, исходя из семейного статуса.
- 'education' и 'education_id': привести текстовые значения к одному регистру.
- 'family_status' и 'family_status_id': также привести текстовые значения к одному нижнему регистру.
- 'gender': есть 1 строка с непроставленным полом, ее можно оставить, не меняя, так как для основных вопросов анализа пол не принципиален.
- 'income_type', 'debt' - не меняем ничего.
- 'purpose' - лемматизируем и переводим в группы по недвижимости, образованию, автокредитам и свадьбам.


## Шаг 2. Предобработка данных

## Обработка пропусков

In [11]:
#приведем к одному регистру образование
data['education'] = data['education'].str.lower()

In [12]:
data.pivot_table(index=['education_id','education'], values='dob_years', aggfunc='count')

,,dob_years
education_id,education,
0,высшее,5260
1,среднее,15233
2,неоконченное высшее,744
3,начальное,282
4,ученая степень,6


Выше мы привели к единому регистру и виду столбец 'education'. Проверили результат, вызвав новую сводную таблицу.

In [13]:
#приведем к одному регистру семейный статус
data.loc[data['family_status_id'] == 4, 'family_status'] = 'не женат / не замужем'

data.pivot_table(index=['family_status_id','family_status'], values='dob_years', aggfunc='count')

,,dob_years
family_status_id,family_status,
0,женат / замужем,12380
1,гражданский брак,4177
2,вдовец / вдова,960
3,в разводе,1195
4,не женат / не замужем,2813


Также скорректировали регистр у одного типа семейного статуса.

In [14]:
#исправляем ошибки в 'children'
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 1

data.groupby('children')['children'].count()

children
0    14149
1     4941
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Также обработали неверные значение в столбце 'children'. Теперь приступим к редактированию столбца про возраст, так как он удет нам нужен для заполнения пропусков в других столбцах.

In [15]:
#расчитаем средний возраст по семейному положению
def average_years_by_famaly_status(status_id):
    average_years = data[(data['dob_years'] != 0)&(data['family_status_id'] == status_id)]['dob_years'].mean()
    average_years = int(average_years)
    return average_years

for value in range(5):
    data.loc[(data['dob_years'] == 0)&(data['family_status_id'] == value), 'dob_years'] = average_years_by_famaly_status(value)
    
data[data['dob_years'] == 0]['dob_years'].count()

0

Мы написали функцию, которая считает для строк, где значение возраст не равно 0, среднее по семейному положению.
Далее мы применили эту функции для замены нулевых значений возраста и проверили, что замена прошла успешно.
Проверим еще, на всякий случай, что не появилось странных значений после применения этой функции.

In [16]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

Вылетов не обнаружено, замены прошли успешно.

In [17]:
#обработка cтолбца 'days_employed'

data.loc[data['days_employed'] > 0, 'days_employed'] = data['days_employed']/24

In [18]:
data[data['days_employed'] > 0]['days_employed'].mean()

15208.51291317786

Мы видим, что среднее по таким значениям изменилось с 365 тыс (выгружали в прошлом параграфе) на 15 тыс. Значит замена прошла успешно.
Теперь приступаем к переводу отрицательных значений в положительные.

In [19]:
data.loc[data['days_employed'] < 0, 'days_employed'] = data['days_employed']*(-1)

data[data['days_employed'] < 0]['days_employed'].count()

0

Теперь мы также перевели все значение по стажу в положительные числа.
Теперь приступим к заполнению пропусков.

In [20]:
def avarage_days_employed(years):
    average_days = data[data['dob_years'] == years]['days_employed'].mean()
    return average_days

avarage_days_employed(60)

11976.07086964992

Мы создали функцию, которая по возрасту выдает средний стаж в днях. И проверили ее на примере. У нее есть узкое место, что если в строчках с пропуском есть такое значение возраста, которого нет в данных без пропусков. Но эти случаи мы проверим после попытки замены основных пропусков.

In [21]:
wrong_lines_content = []

for value in range(data['dob_years'].max()+1):
    try:
        data.loc[(data['days_employed'].isnull())&(data['dob_years'] == value), 'days_employed'] = avarage_days_employed(value)
    except:  
        wrong_lines_content.append(value)
        
print(wrong_lines_content)

[]


Видим, что алгоритм по замене сработал на по всем строкам с пустыми значениями. Перепроверим, что не осталось пустот. А также проверим выборочно, что по другим изначально не пустым значениям с одним и тем же возрастом мы не заменили стаж на одинаковое среднее значение.

In [22]:
data[data['days_employed'].isnull()].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [23]:
data[data['dob_years'] == 42].head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
61,0,2986.202120,42,среднее,1,гражданский брак,1,F,сотрудник,0,173987.472929,покупка недвижимости
89,0,72.625785,42,среднее,1,гражданский брак,1,F,компаньон,0,49463.919380,сделка с подержанным автомобилем
130,0,897.322806,42,среднее,1,женат / замужем,0,F,компаньон,0,95845.744707,операции со своей недвижимостью
215,0,79.880350,42,высшее,0,не женат / не замужем,4,F,компаньон,0,260465.394500,операции с недвижимостью
324,0,7694.849832,42,высшее,0,женат / замужем,0,M,госслужащий,0,320700.927536,операции с коммерческой недвижимостью
330,1,1444.250750,42,среднее,1,не женат / не замужем,4,F,компаньон,0,175328.605163,покупка жилья
351,1,6799.624714,42,среднее,1,женат / замужем,0,F,госслужащий,0,290780.735919,покупка коммерческой недвижимости
386,0,2093.262819,42,среднее,1,женат / замужем,0,M,сотрудник,0,136525.031056,покупка жилья
527,0,998.045061,42,высшее,0,гражданский брак,1,M,сотрудник,0,71263.231167,свадьба


Пропуски заполнены корректно: по этому столбцу их больше не осталось, а также видим, что мы не затронули другие значения.

Теперь перейдем к работе над столбцом с доходом.
В нем мы будем брать медиану, так как есть чрезмерновысокие значения. А также, если на стаж больше всего влияет именно возраст. То доход еще сильно зависит и от рода деятельности.


In [24]:
#обработка толбца 'total_income'
def median_of_income(years,income_type):
    group = data[(data['dob_years'] == years)&(data['income_type'] == income_type)]
    median_income = group.sort_values(by=['total_income'], ascending=False)['total_income'].median()
    return median_income

median_of_income(60,'сотрудник')

142156.95778283276

In [25]:
median_of_income(42,'сотрудник')

151702.86228896093

По значению выше проверим заполнение, так как в строке 3979 как был такой пропуск.

Мы написали функции для замены пропусков по доходу. Теперь применим ее для заполнения этих пропусков.

In [27]:
wrong_lines_content = []
income_type_set = data['income_type'].unique()

for element in income_type_set:
    for value in range(data['dob_years'].max()+1):
        try:
            data.loc[(data['total_income'].isnull())&(data['dob_years'] == value)&(data['income_type'] == element), 'total_income'] = median_of_income(value, element)
        except:
            wrong_lines_content.append(value)


In [28]:
data[data['total_income'].isnull()].count()

children            4
days_employed       4
dob_years           4
education           4
education_id        4
family_status       4
family_status_id    4
gender              4
income_type         4
debt                4
total_income        0
purpose             4
dtype: int64

In [29]:
data.loc[3979]

children                                       5
days_employed                            2962.62
dob_years                                     42
education                                среднее
education_id                                   1
family_status                   гражданский брак
family_status_id                               1
gender                                         M
income_type                            сотрудник
debt                                           0
total_income                              151703
purpose             на покупку своего автомобиля
Name: 3979, dtype: object

После замены мы видим, что не по всем пропускам нашлись релевантные группы. 4 строки остались незаполнены. Посмотрим на них внимательнее.

С другой стороны выборочная проверка показала правильный результата замены. Строка 3979 была с пропусками. После замены в значении доход ровно то значение медианы, которое расчиталось ля этой группы возраст 42 и сотрудник.

In [30]:
data[data['total_income'].isnull()].head(4)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3619,0,1026.405485,24,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,покупка своего жилья
5936,0,9381.056663,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости
17374,0,14369.789848,69,высшее,0,не женат / не замужем,4,F,компаньон,0,NaN,автомобили
18175,2,1677.790261,31,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,свадьба


Эти случаи уникальные. В данном случае отличие от усредненных групп заключается в возрасте или рекости профессии. Для них замены сделаем только по медиане по роду деятельности.

In [31]:
def median_of_income_by_only_type(income_type):
    group = data[data['income_type'] == income_type]
    median_income = group.sort_values(by=['total_income'], ascending=False)['total_income'].median()
    return median_income

for element in income_type_set:
    try:
        data.loc[(data['total_income'].isnull())&(data['income_type'] == element), 'total_income'] = median_of_income_by_only_type(element)
    except:
            wrong_lines_content.append(value)

In [32]:
data[data['total_income'].isnull()].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Мы избавились от всех пропусков.

### Вывод

Итого:
- мы привели к единому регистру и формату текстовые значения в столбцах: education и family_status
- мы устранили критичные ошибки (количество детей -1, 20, возраст 0 лет, отрицательные и завышенные в 24 раза дни работы) с следующих столбцах: children, dob_years, days_employed
- мы заполнили все пропуски в столбцах (2174), используя среднее и медиану по различных группам клиентов: days_employed и total_income

Теперь нужно еще раз посмотреть на заполненную таблицу, проверить типы данных и дубликаты.

## Замена типа данных

In [33]:
data.info()
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


До изменения данных память, которую занимают данные - 2.0+ MB.

Все текстовые значения имеют фомат - object. Пока их не будем трогать.

Но в некоторых числовых значениях есть вещественный тип (float64). Если для дохода это возможно, так как округления после вычета налогов могут давать такую погрешность, то для стажа в днях это некорректно. Нужно перевести это в целый тип данный.

In [36]:
data['days_employed']=data['days_employed'].astype('int')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


По чиcловым типам данных также можно сэкономить память и перевести их из int64 в bool, int8, int16 и другие. Вызовем метод describe() для изучения применения потенциальных форматов.

In [37]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.476190,0.750737,0.000000,0.000000,0.000000,1.000000,5.000000e+00
days_employed,21525.0,4652.442091,5221.475370,24.000000,1013.000000,2342.000000,5750.000000,1.838800e+04
dob_years,21525.0,43.494402,12.220726,19.000000,34.000000,43.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,21525.0,165271.393444,98110.258704,20667.263793,107344.262632,144364.522847,196001.459765,2.265604e+06


Если изучить диапазоны наших значений, то можно выбрать следующие потенциальные форматы для столбцов:
- 'children' диапазон от 0 до 5, тип int8.
- 'days_employed' диапазон от 24 до 18 тыс, тип int16 (подходит, так как диапазон до 32 тыс).
- 'dob_years' диапазон от 19 до 75, тип int8.
- 'education_id' диапазон от 0 до 4, тип int8.
- 'family_status_id' диапазон от 0 до 4, тип int8.
- 'debt' диапазон от 0 до 1, тип int8 или bool. Но они занимают одинаковое число памяти, можно выбрать любой.
- 'total_income' от 20 тыс до 2 с чем-то млн. Это 6 бит экспоненты. Можно заменить на float32.


In [38]:
data['children'] = np.int8(data['children'])

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int8
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), int8(1), object(5)
memory usage: 1.8+ MB


Мы попробовали заменить тип на 1 столбце и проверили как прошла замена: тип изменился, память уже 1.8+ MB.

Продолжим с другими столбцами.

In [40]:
data['days_employed'] = np.int16(data['days_employed'])
data['dob_years'] = np.int8(data['dob_years'])
data['education_id'] = np.int8(data['education_id'])
data['family_status_id'] = np.int8(data['family_status_id'])
data['debt'] = np.bool_(data['debt'])
data['total_income'] = np.float32(data['total_income'])

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int8
days_employed       21525 non-null int16
dob_years           21525 non-null int8
education           21525 non-null object
education_id        21525 non-null int8
family_status       21525 non-null object
family_status_id    21525 non-null int8
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null bool
total_income        21525 non-null float32
purpose             21525 non-null object
dtypes: bool(1), float32(1), int16(1), int8(4), object(5)
memory usage: 1.0+ MB


In [42]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.476190,0.750737,0.000000,0.000000,0.000000,1.000000,5.0
days_employed,21525.0,4652.442091,5221.475370,24.000000,1013.000000,2342.000000,5750.000000,18388.0
dob_years,21525.0,43.494402,12.220726,19.000000,34.000000,43.000000,53.000000,75.0
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.0
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.0
total_income,21525.0,165271.406250,98110.257812,20667.263672,107344.265625,144364.515625,196001.453125,2265604.0


Выше мы проверили, что наши максимальные и минимальные значения не перезаписались в 0 и мы правильно выбрали форматы. 

Если смотреть на обновленную память, то она сильно сократилась и стала 1.0+ MB (в два раза меньше).

Далее приступим к обработке текстовых столбцов (object). Для всех столбцов нам поможет категорийный тип. Так как эффективен, если хотя бы половина значений повторяется. У нас же самый разнообразным является 'purpose', но у него около 50-100 уникальных значений на 21 тыс строк.

In [46]:
data['education'] = data['education'].astype('category')
data['family_status'] = data['family_status'].astype('category')
data['gender'] = data['gender'].astype('category')
data['income_type'] = data['income_type'].astype('category')
data['purpose'] = data['purpose'].astype('category')

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int8
days_employed       21525 non-null int16
dob_years           21525 non-null int8
education           21525 non-null category
education_id        21525 non-null int8
family_status       21525 non-null category
family_status_id    21525 non-null int8
gender              21525 non-null category
income_type         21525 non-null category
debt                21525 non-null bool
total_income        21525 non-null float32
purpose             21525 non-null category
dtypes: bool(1), category(5), float32(1), int16(1), int8(4)
memory usage: 338.9 KB


In [48]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875.640625,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080.015625,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885.953125,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628.562500,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616.078125,сыграть свадьбу


Замены прошли успешно, память сократилась до 338.9 KB.

### Вывод

Мы произвели проверку типов данных. 

После для стажа в днях ('days_employed') мы поменяли тип на целочисленный и проверили результат замены. Мы использовали метод astype, так как жто позволило быстро перевести из одного числового формата наши данные.

Далее для всех числовых форматов мы подорали новый тип данных, который позволил сократить размер используемой памяти: bool, int8, int16 и float32. Для этого мы импортировали и использовали билиотеку NumPy.


До преобразования типов память была занята на 2.0+ MB. Если смотреть на обновленную память, то она сильно сократилась и стала 1.0+ MB (в два раза меньше).

После мы поработали на текстовыми значениями, переведя их в категорийный тип и память сократилась до 338.9 KB.

## Обработка дубликатов

In [49]:
data.duplicated().sum()

72

Выше мы увидели, что в нашей таблице 72 дубликата, это меньше половины процента от всех строк. В данном случае удалим данные строки.

In [50]:
data = data.drop_duplicates().reset_index(drop=True)

In [51]:
data.duplicated().sum()

0

In [52]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
children            21453 non-null int8
days_employed       21453 non-null int16
dob_years           21453 non-null int8
education           21453 non-null category
education_id        21453 non-null int8
family_status       21453 non-null category
family_status_id    21453 non-null int8
gender              21453 non-null category
income_type         21453 non-null category
debt                21453 non-null bool
total_income        21453 non-null float32
purpose             21453 non-null category
dtypes: bool(1), category(5), float32(1), int16(1), int8(4)
memory usage: 337.7 KB


Дубликаты успешно удалены.

### Вывод

Мы нашли 72 строки дубликата, что составляем меньше половины процента от всех строк. Мы использовали метод duplicated() в комбинаций с sum().

Дубликаты могли появиться из-за дублирования информации в разных источниках данных (переслали сначал часть с одного филиала, а потом эти данные попали в общие данные кластера филиалов), также человек мог заполняя заявку электронно нажать на кнопку отправки несколько раз.

Дубликаты были успешно удалены с помощью метода drop_duplicates(), после проверки мы видим, что теперь число строк 21453.

## Лемматизация

Лемматизация поможет нам для формирования более четких целей кредита, которые мы запишем в отдельном столбце.
Ранее мы выделили следующие основные направления:
- жилье/недвижимость
- образование
- автокредит
- свадьба

Давайте напишем функцию, которая при наличии определенных лемм выдает определенный тип кредита.

In [54]:
#создаем функцию для категоризации с помощью лемматизации
def type_of_purpose(text):
    lemmas = m.lemmatize(text)
    if 'свадьба' in lemmas:
        return 'свадьба'
    elif 'недвижимость' in lemmas:
        return 'недвижимость'
    elif 'жилье' in lemmas:
        return 'недвижимость'
    elif 'жильё' in lemmas:
        return 'недвижимость'
    elif 'автомобиль' in lemmas:
        return 'автокредит'
    elif 'машина' in lemmas:
        return 'автокредит'
    elif 'образование' in lemmas:
        return 'образование'
    else:
        return 'другое'
        
#добавляем новый столбец с новой категорией
data['purpose_type'] = data['purpose'].apply(type_of_purpose)

In [55]:
data.info()
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 13 columns):
children            21453 non-null int8
days_employed       21453 non-null int16
dob_years           21453 non-null int8
education           21453 non-null category
education_id        21453 non-null int8
family_status       21453 non-null category
family_status_id    21453 non-null int8
gender              21453 non-null category
income_type         21453 non-null category
debt                21453 non-null bool
total_income        21453 non-null float32
purpose             21453 non-null category
purpose_type        21453 non-null object
dtypes: bool(1), category(5), float32(1), int16(1), int8(4), object(1)
memory usage: 505.3+ KB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875.640625,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080.015625,приобретение автомобиля,автокредит
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885.953125,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628.562500,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616.078125,сыграть свадьбу,свадьба


Видим, что столбец успешно добавился. Теперь проверим не появилось ли значение "другое" в этом столбце, так как мы его указывали как остаточное, если основные слова не найдутся.

In [56]:
 data['purpose_type'].value_counts()

недвижимость    10811
автокредит       4306
образование      4013
свадьба          2323
Name: purpose_type, dtype: int64

Видим, что слов без категорий не осталось, у каждой строки стоит 1 из 4 целей, которые мы определили. Теперь осталось поменять тип столбца на категориальный и можно переходить к дальнейшим шагам.

In [57]:
data['purpose_type'] = data['purpose_type'].astype('category')

In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 13 columns):
children            21453 non-null int8
days_employed       21453 non-null int16
dob_years           21453 non-null int8
education           21453 non-null category
education_id        21453 non-null int8
family_status       21453 non-null category
family_status_id    21453 non-null int8
gender              21453 non-null category
income_type         21453 non-null category
debt                21453 non-null bool
total_income        21453 non-null float32
purpose             21453 non-null category
purpose_type        21453 non-null category
dtypes: bool(1), category(6), float32(1), int16(1), int8(4)
memory usage: 358.9 KB


### Вывод

Мы успешно импортировали библиотеку для лемматизации данных из столбца с целями кредита.
Для дальнейшей классификации данных и работы мы создали столбец с более общей целью. Для этого мы создали отдельную функцию, котогая искала ключевые слова в леммах.

## Категоризация данных

В теме выше мы уже проклассифицировали данные, исходя из типов целей кредита.

Но также будет полезно и удобно проклассифицировать данные по критериям, относящимся к самим клиентам. Часть данных по ним и так категоризирована в общие группы (такие как семейное положение, род деятельности). Но вот возраст, к примеру носит не однородный характер. Создадим категории данных на его основе.

Также для целей проекта будет удобно создать маркет о наличии детей и категории по уровню дохода.

In [59]:
#создаем функцию для категоризации по возрасту
def age_group(years):
    if years < 36:
        return 'молодежь'
    elif years < 55:
        return 'взрослые'
    else:
        return 'пожилые'
    
#добавляем новый столбец с новой категорией
data['age_group'] = data['dob_years'].apply(age_group)
data['age_group'] = data['age_group'].astype('category')

data.info()
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 14 columns):
children            21453 non-null int8
days_employed       21453 non-null int16
dob_years           21453 non-null int8
education           21453 non-null category
education_id        21453 non-null int8
family_status       21453 non-null category
family_status_id    21453 non-null int8
gender              21453 non-null category
income_type         21453 non-null category
debt                21453 non-null bool
total_income        21453 non-null float32
purpose             21453 non-null category
purpose_type        21453 non-null category
age_group           21453 non-null category
dtypes: bool(1), category(7), float32(1), int16(1), int8(4)
memory usage: 379.9 KB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type,age_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875.640625,покупка жилья,недвижимость,взрослые
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080.015625,приобретение автомобиля,автокредит,взрослые
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885.953125,покупка жилья,недвижимость,молодежь
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628.562500,дополнительное образование,образование,молодежь
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616.078125,сыграть свадьбу,свадьба,взрослые


In [60]:
#создаем функцию для маркера о наличии детей
def mark_children(children):
    if children > 0:
        return 'да'
    else:
        return 'нет'
    
#добавляем новый столбец с новой категорией
data['mark_children'] = data['children'].apply(mark_children)
data['mark_children'] = data['mark_children'].astype('category')

data.info()
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 15 columns):
children            21453 non-null int8
days_employed       21453 non-null int16
dob_years           21453 non-null int8
education           21453 non-null category
education_id        21453 non-null int8
family_status       21453 non-null category
family_status_id    21453 non-null int8
gender              21453 non-null category
income_type         21453 non-null category
debt                21453 non-null bool
total_income        21453 non-null float32
purpose             21453 non-null category
purpose_type        21453 non-null category
age_group           21453 non-null category
mark_children       21453 non-null category
dtypes: bool(1), category(8), float32(1), int16(1), int8(4)
memory usage: 401.0 KB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type,age_group,mark_children
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875.640625,покупка жилья,недвижимость,взрослые,да
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080.015625,приобретение автомобиля,автокредит,взрослые,да
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885.953125,покупка жилья,недвижимость,молодежь,нет
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628.562500,дополнительное образование,образование,молодежь,да
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616.078125,сыграть свадьбу,свадьба,взрослые,нет


In [62]:
#создаем функцию для маркера о наличии детей
def children_group(children):
    if children > 1:
        return 'многодетные'
    elif children == 1:
        return 'один ребенок'
    else:
        return 'без детей'
    
#добавляем новый столбец с новой категорией
data['children_group'] = data['children'].apply(children_group)
data['children_group'] = data['children_group'].astype('category')

data.info()
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 16 columns):
children            21453 non-null int8
days_employed       21453 non-null int16
dob_years           21453 non-null int8
education           21453 non-null category
education_id        21453 non-null int8
family_status       21453 non-null category
family_status_id    21453 non-null int8
gender              21453 non-null category
income_type         21453 non-null category
debt                21453 non-null bool
total_income        21453 non-null float32
purpose             21453 non-null category
purpose_type        21453 non-null category
age_group           21453 non-null category
mark_children       21453 non-null category
children_group      21453 non-null category
dtypes: bool(1), category(9), float32(1), int16(1), int8(4)
memory usage: 422.0 KB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type,age_group,mark_children,children_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875.640625,покупка жилья,недвижимость,взрослые,да,один ребенок
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080.015625,приобретение автомобиля,автокредит,взрослые,да,один ребенок
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885.953125,покупка жилья,недвижимость,молодежь,нет,без детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628.562500,дополнительное образование,образование,молодежь,да,многодетные
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616.078125,сыграть свадьбу,свадьба,взрослые,нет,без детей


In [63]:
#создаем функцию для категорий по уровню дохода
def level_of_income(total_income):
    if total_income < 100001:
        return '1_низкий'
    elif total_income < 150001:
        return '2_средний'
    elif total_income < 250001:
        return '3_высокий'
    else:
        return '4_очень высокий'
    
#добавляем новый столбец с новой категорией
data['level_of_income'] = data['total_income'].apply(level_of_income)
data['level_of_income'] = data['level_of_income'].astype('category')

data.info()
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 17 columns):
children            21453 non-null int8
days_employed       21453 non-null int16
dob_years           21453 non-null int8
education           21453 non-null category
education_id        21453 non-null int8
family_status       21453 non-null category
family_status_id    21453 non-null int8
gender              21453 non-null category
income_type         21453 non-null category
debt                21453 non-null bool
total_income        21453 non-null float32
purpose             21453 non-null category
purpose_type        21453 non-null category
age_group           21453 non-null category
mark_children       21453 non-null category
children_group      21453 non-null category
level_of_income     21453 non-null category
dtypes: bool(1), category(10), float32(1), int16(1), int8(4)
memory usage: 443.2 KB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type,age_group,mark_children,children_group,level_of_income
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875.640625,покупка жилья,недвижимость,взрослые,да,один ребенок,4_очень высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080.015625,приобретение автомобиля,автокредит,взрослые,да,один ребенок,2_средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885.953125,покупка жилья,недвижимость,молодежь,нет,без детей,2_средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628.562500,дополнительное образование,образование,молодежь,да,многодетные,4_очень высокий
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616.078125,сыграть свадьбу,свадьба,взрослые,нет,без детей,3_высокий


Давайте также посмотрим сколько строк в наших категориях.

In [64]:
data.groupby('level_of_income')['level_of_income'].count()

level_of_income
1_низкий           4468
2_средний          7102
3_высокий          7067
4_очень высокий    2816
Name: level_of_income, dtype: int64

In [65]:
data.groupby('age_group')['age_group'].count()

age_group
взрослые    10086
молодежь     6583
пожилые      4784
Name: age_group, dtype: int64

In [66]:
data.groupby('mark_children')['mark_children'].count()

mark_children
да      7363
нет    14090
Name: mark_children, dtype: int64

In [67]:
data.groupby('children_group')['children_group'].count()

children_group
без детей       14090
многодетные      2432
один ребенок     4931
Name: children_group, dtype: int64

In [68]:
data.groupby('purpose_type')['purpose_type'].count()

purpose_type
автокредит       4306
недвижимость    10811
образование      4013
свадьба          2323
Name: purpose_type, dtype: int64

### Вывод

Выше мы категоризовали данные, исходя из возраста клиента: 'молодежь'(6583), 'взрослые'(10086), 'пожилые'(4784).

Ранее мы классифицировали данные, исходя из типа цели кредита: 'свадьба'(2323), 'автокредит'(4306), 'недвижимость'(10811), 'образование'(4013).

Также был создан отдельный маркер на наличие детей (7363 строки с детьми и 14090 без). И группы, где выделили многодетных родителей (2432) и с 1 ребенком (4931).

А также уровни заработка (от низкого до очень высокого). В уровнях заработка в название было добавлено число, чтобы при изучении результатов данные выстаривались в возрастабщем порядке по заработку.

Порядок цифр во всех группах одинаковый. Конечно где-то есть явные группы лидеры, но это связано со спецификой данных (кридит чаще берут взрослые и недвижимость полулярная цель).

Также для анализа и формирования финальных выводов можно пользоваться категориями, которые изначально были нам даны (семейное положение, образование, тип занятости).

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Чтобы ответить на этот вопрос сначала посмотрим на то, есть ли такие случаи в этих группах. Сформируем сводную таблицу по маркеру о наличии детей и маркеру задолженности.

In [69]:
data.pivot_table(index='mark_children', columns='debt', values='purpose', aggfunc='count')

debt,False,True
mark_children,,
да,6685,678
нет,13027,1063


Как мы видим случаи невозврата кредита есть в обеих группах: у кого есть дети и нет.

Посмотрим более внимательно, какая это часть от общих заявок по кредитам в каждой группе.
Для это напишем специаьную функцию, которой будем пользоваться и для других кейсов.

In [70]:
#напишем функцию для определния процента случаев, когда кредит вернули в срок
def parts_debt(column):
    debt_number = data.groupby(column)['debt'].count()
    debt_negative_sum = data.groupby(column)['debt'].sum()
    debt_positive_sum = debt_number - debt_negative_sum
    debt_negative_part = debt_negative_sum/debt_number 
    debt_positive_part = debt_positive_sum/debt_number
    return 'Вернули вовремя ' + (100. * debt_positive_part).round(1).astype(str) + '%, Вернули с опозданием ' + (100. * debt_negative_part).round(1).astype(str) + '%'


#применим функцию к группе маркер наличия детей
parts_debt('mark_children')

mark_children
да     Вернули вовремя 90.8%, Вернули с опозданием 9.2%
нет    Вернули вовремя 92.5%, Вернули с опозданием 7.5%
Name: debt, dtype: object

Видим, что на 1,7%, люди, у которых есть дети, чаще опаздывают с возвратом кредита.

Посмотрим, как это распределено, если смотреть на количество детей более детально.

In [71]:
parts_debt('children_group')

children_group
без детей       Вернули вовремя 92.5%, Вернули с опозданием 7.5%
многодетные     Вернули вовремя 90.7%, Вернули с опозданием 9.3%
один ребенок    Вернули вовремя 90.8%, Вернули с опозданием 9.2%
Name: debt, dtype: object

Видим, что показатели у детей с несколькими детьми и одним почти не отличаются при группировке: 90,8% и 90,7%.

In [72]:
parts_debt('children')

children
0     Вернули вовремя 92.5%, Вернули с опозданием 7.5%
1     Вернули вовремя 90.8%, Вернули с опозданием 9.2%
2     Вернули вовремя 90.5%, Вернули с опозданием 9.5%
3     Вернули вовремя 91.8%, Вернули с опозданием 8.2%
4     Вернули вовремя 90.2%, Вернули с опозданием 9.8%
5    Вернули вовремя 100.0%, Вернули с опозданием 0.0%
Name: debt, dtype: object

Видим, что только наличие 5 детей имеет значительно положительный перевес, в остальных же случаят тенденция сохраняется.

Итого: наличие детей отрицательно влияет на возврат кредита, хоть и не значительно. Люди, у которых нет детей имеют опоздания только в 7,5% случаев. А люди с детьми в 9,2%.
При этом выялена гипотеза, что наличие пяти детей влияет исключительно положительно на сроки погашения кредита. Но чтобы ее окончательно подтвердить не хватает данных, так как таких случаев только 9.

### Вывод

Чтобы ответить на вопрос ниже воспользуем функцией parts_debt(column), которую мы писали выше.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [73]:
parts_debt('family_status')

family_status
в разводе                Вернули вовремя 92.9%, Вернули с опозданием 7.1%
вдовец / вдова           Вернули вовремя 93.4%, Вернули с опозданием 6.6%
гражданский брак         Вернули вовремя 90.7%, Вернули с опозданием 9.3%
женат / замужем          Вернули вовремя 92.5%, Вернули с опозданием 7.5%
не женат / не замужем    Вернули вовремя 90.2%, Вернули с опозданием 9.8%
Name: debt, dtype: object

Прослеживается интересная законмерность, что люди, которые состояли или состоят в официальном браке чаще отдают кредит вовремя, более чем в 92,5%.

А вот те, кто не состоит в официалном браке чаще зарерживают выплаты примерно, и успевают в срок только в среднем в 90,5% случаев.
Итого: наличие официального семейного статуса в прошлом или в настояшем положительно влияет на вероятность возвращения кредита.

### Вывод

Чтобы ответить на вопрос ниже воспользуем функцией parts_debt(column), которую мы писали выше.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [74]:
parts_debt('level_of_income')

level_of_income
1_низкий           Вернули вовремя 92.1%, Вернули с опозданием 7.9%
2_средний          Вернули вовремя 91.3%, Вернули с опозданием 8.7%
3_высокий          Вернули вовремя 91.8%, Вернули с опозданием 8.2%
4_очень высокий    Вернули вовремя 93.1%, Вернули с опозданием 6.9%
Name: debt, dtype: object

Тут мы наблюдаем интересную закономерность:
- среди людей с очень высоким доходом процент возврата вовремя очень высокий (93,1%)
- но у людей среднего и просто высокого достатка уровень дисциплина не такая хорошая (91,3% и 91,8%).
- а вот у людей с низким доходом дисциплина возврата опять повышается (92,1%).

Возможно это связано с тем, что люди с низким доходом берут кредит только в крайних случаях и из-за этого отвественно относятся к его погашению.
Другие же группы чувствуют себя более безопасно и берут кредиты чаще, и от этого ведут себя более дисциплинированно.

### Вывод

Чтобы ответить на вопрос ниже воспользуем функцией parts_debt(column), которую мы писали выше.

- Как разные цели кредита влияют на его возврат в срок?

In [75]:
parts_debt('purpose_type')

purpose_type
автокредит      Вернули вовремя 90.6%, Вернули с опозданием 9.4%
недвижимость    Вернули вовремя 92.8%, Вернули с опозданием 7.2%
образование     Вернули вовремя 90.8%, Вернули с опозданием 9.2%
свадьба         Вернули вовремя 92.0%, Вернули с опозданием 8.0%
Name: debt, dtype: object

Видим, что кредиты по недвижимости возвращают лучше всего (92,8%). Также хорошо возвращают кредиты на свадьбу (92%). Но мы то с вами уже знаем, что брак магическим образом влияет на  способность отдавать кредит. Чему мы и видим опять подтверждение.

А вот кредит на образование и автокредит отдают хуже. Только в 90,8% и 90,6% случаев соблюдаются сроки.

### Вывод

Также посмотрим на то, как распределены должники по возрастным группам и роду деятельности.

In [76]:
parts_debt('age_group')

age_group
взрослые     Вернули вовремя 92.3%, Вернули с опозданием 7.7%
молодежь    Вернули вовремя 89.4%, Вернули с опозданием 10.6%
пожилые      Вернули вовремя 94.5%, Вернули с опозданием 5.5%
Name: debt, dtype: object

А тут и спрятался самый весомый вывод. Молодежь совсем плохо соблюдает расписание по возвратам (89,4%), взрослые ведут себя достаточно дисциплинировано (92,3%), а лучше всех себя проявляют люди старше 55 лет (94,5%).

In [77]:
parts_debt('income_type')

income_type
безработный        Вернули вовремя 50.0%, Вернули с опозданием 50.0%
в декрете          Вернули вовремя 0.0%, Вернули с опозданием 100.0%
госслужащий         Вернули вовремя 94.1%, Вернули с опозданием 5.9%
компаньон           Вернули вовремя 92.6%, Вернули с опозданием 7.4%
пенсионер           Вернули вовремя 94.4%, Вернули с опозданием 5.6%
предприниматель    Вернули вовремя 100.0%, Вернули с опозданием 0.0%
сотрудник           Вернули вовремя 90.4%, Вернули с опозданием 9.6%
студент            Вернули вовремя 100.0%, Вернули с опозданием 0.0%
Name: debt, dtype: object

Видим очевидные группы риска: безработные (50%), в декрете (0%).

Так и наименее рискованные: студент (100%), предприниматель (100%), пенсионер (94,4%), госслужащий (94,1%), компаньон (92.6%).
Сотрудники уже чаще групп выше не соблюдают сроки (90,4%).

## Шаг 4. Общий вывод

Итого по результатам анализа данных можно выявить следюущие закономерности:
- Видим, что на 1,7%, люди, у которых есть дети, чаще опаздывают с возвратом кредита. (92,5% возвратов в срок у клиентов без детей и 90,8% с детьми). Причем при наличии 1 ребенка или нескольких клиенты ведут себя примерно одинаково (90,8% и 90,7%).
- Есть потенциальная интересная гипотеза, что у людей с 5 детьми наоборот очень хорошая история погашения кредитов. Но данных проверить ее сейчас недостаточно (9 кейсов). Но возможно это именно то число детей, с которым общая дисциплина вынужено повышается, что влияет и на погашение крелитов.
- Те люди, которые состоят или состояли в официальном браке также чаще возвращают кредиты без задержек (92,5% и 90,5%). Эта гипотеза также подтверждается интересным наблюдением, что возврат кредит на свадьбу также очень высок (92%). А ведь это прямое свидетельство того, что эти люди собираются в официальный брак.
- Зависимость от дохода наблюдается следующая: 
    - среди людей с очень высоким доходом процент возврата вовремя очень высокий (93,1%)
    - но у людей среднего и просто высокого достатка уровень дисциплина не такая хорошая (91,3% и 91,8%).
    - а вот у людей с низким доходом дисциплина возврата опять повышается (92,1%).
  Вероятно это свяано с тем, что люди с низким доходом берут кредит только в крайних случаях и из-за этого отвественно относятся к его погашению. Другие же группы чувствуют себя более безопасно и берут кредиты чаще, и от этого ведут себя более дисциплинированно.
- Кредиты по недвижимости возвращают хорошо (92,8%), как и кредиты на свадьбу (92%). А вот кредит на образование и автокредит отдают хуже. Только в 90,8% и 90,6% случаев соблюдаются сроки.
- Группы риска: безработные (50%), в декрете (0%).
- Наименее рискованные: студент (100%), предприниматель (100%), пенсионер (94,4%), госслужащий (94,1%), компаньон (92.6%). Сотрудники уже чаще групп выше не соблюдают сроки (90,4%).
- Молодежь плохо соблюдает расписание по возвратам (89,4%), взрослые ведут себя достаточно дисциплинировано (92,3%), а лучше всех себя проявляют люди старше 55 лет (94,5%).

Подводя заключение попробуем сформировать примеры портретов наиболее дисциплинированных и нет клиентов:
- Идеальный клиент:  Человек возраста 55 лет, предприниматель, недавно вступил в брак, но пока без детей. Хочет расширить свою жилплощадь. 
- Рискованный клиент: Человек возраста 25 лет, сотрудник фирмы, живущий в гражданском браке с ребенком. Берет кредит на автомобиль.

Спасибо за чтение и изучение проекта!

Виктория

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.